In [1]:
from utils.BoW_data import load_dataframe as ld

In [2]:
mlb_tags_train, mlb_words_train, df_train = ld()

TypeError: can only concatenate list (not "str") to list

In [ ]:
mlb_words_train.classes_[0]

In [ ]:
df_train.head()

In [ ]:
_,_, df_test = ld('test')

In [ ]:
df_test.head()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
num_classes_tags = len(mlb_tags_train.classes_)
num_classes_words = len(mlb_words_train.classes_)

In [ ]:
def word_list_to_vector(mlb,word_list,num_classes):
    r = [0]*num_classes
    for i,w in enumerate(mlb.classes_):
        if w in word_list:
            r[i] = 1
    return r

In [ ]:
def tags_list_to_vector(mlb,tags_list,num_classes):
    r = [0]*num_classes
    for i,t in enumerate(mlb.classes_):
        if t in tags_list:
            r[i] = 1
    return r

In [ ]:
test_tags = df_test['tags_values'].apply(lambda x: tags_list_to_vector(mlb_tags_train,x,num_classes_tags))
test_vectors = df_test['word_list'].apply(lambda x: word_list_to_vector(mlb_words_train,x,num_classes_words))

In [ ]:
clf = MultinomialNB()
x = df_train['word_vector'].tolist()
y = df_train['image_file'].tolist()
clf.fit(x, y)

In [ ]:
train_tags = df_train['tags_vec'].tolist()
test_img_tags = list(zip(df_test['image_file'].tolist(),df_test['tags_vec'].tolist()))

In [ ]:
def euclidean_distance(v1,v2):
    sum = 0
    for x,y in zip(v1,v2):
        sum += (x-y)**2
    return sum**0.5

In [ ]:
def nearest20(training_tag,test_tags):
    nearest_test_neighbors = []
    for t in test_tags:
        nearest_test_neighbors.append((t[0],euclidean_distance(training_tag,t[1])))
    nearest_test_neighbors.sort(key=lambda x: x[1])
    return nearest_test_neighbors[:20]

In [ ]:
predictions = df_train['tags_vec'].apply(lambda tags: nearest20(tags,test_img_tags))

In [ ]:
print(predictions[0])

In [ ]:
train_to_test_dict = {}
for idx, image_file in enumerate(df_train['image_file'].tolist()):
    train_to_test_dict[image_file] = predictions[idx]

In [ ]:
predicted_training_classes = clf.predict(test_vectors.tolist())

In [ ]:
predicted_training_classes[0]

In [ ]:
predicted_test_classes = []
for p in predicted_training_classes:
    predicted_test_classes.append(train_to_test_dict[p])

In [ ]:
predicted_test_classes[1]

In [ ]:
submission_images_list = [[p[0].split('/')[1] for p in l] for l in predicted_test_classes]

In [ ]:
submission_images_string = []
for e in submission_images_list:
    s = ""
    for i in e:
        s = s + i + " "
    submission_images_string.append(s)

In [ ]:
print(submission_images_string[:2])

In [ ]:
image_indeces = df_test['image_file'].tolist()
image_indeces = [(f.split('/')[1]).split('.')[0] + ".txt" for f in image_indeces]

In [ ]:
image_indeces[:10]

In [ ]:
submission_list = list(zip(image_indeces,submission_images_string))
submission_list.sort(key=lambda x: int(x[0].split('.')[0]))
submission_list[0]

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
submission_df = pd.DataFrame(np.array(submission_list),columns=['Descritpion_ID','Top_20_Image_IDs'])

In [ ]:
submission_df.to_csv('submission.csv',index=False)

In [ ]:
x = [[1,2],[3,4]]
r = []
for e in x:
    r = r + e